# 📊 ToyaSapo 問い合わせデータ分析（自動化版）

## 🚀 自動化のポイント
- **サービスアカウントキー**: Google Driveから自動読み込み
- **メールデータ**: Firestoreから直接取得（CSVアップロード不要）

## 📋 初回セットアップ（1回だけ）
1. Google Driveの `001_PBL_チームD_AI4th` フォルダにサービスアカウントキーを配置
2. ファイル名: `toyasapo-firebase-adminsdk-fbsvc-93683c6f93.json`

## 📋 毎回の使い方
1. このノートブックを開く
2. 「ランタイム」→「すべてのセルを実行」
3. 完了！

## 1. セットアップ

In [ ]:
# 必要なライブラリをインストール
!apt-get -y install fonts-ipafont-gothic > /dev/null 2>&1
!pip install janome -q
!pip install japanize-matplotlib -q
!pip install firebase-admin -q

print("✅ インストール完了")

✅ インストール完了


In [ ]:
# ライブラリのインポート
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
from collections import Counter
from janome.tokenizer import Tokenizer
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'IPAPGothic'
japanize_matplotlib.japanize()
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ ライブラリ読み込み完了")

✅ ライブラリ読み込み完了


## 2. Google Drive マウント & Firebase初期化

サービスアカウントキーの場所:
`マイドライブ/001_PBL_チームD_AI4th/toyasapo-firebase-adminsdk-fbsvc-93683c6f93.json`

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive マウント完了")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive マウント完了


In [ ]:
# Firebase初期化（Google Driveからキーを読み込み）
import firebase_admin
from firebase_admin import credentials, firestore
import os

# サービスアカウントキーのパス
KEY_PATH = '/content/drive/MyDrive/001_PBL_チームD_AI4th/toyasapo-firebase-adminsdk-fbsvc-93683c6f93.json'

# ファイル存在チェック
if not os.path.exists(KEY_PATH):
    print("❌ エラー: firebase-key.json が見つかりません")
    print("")
    print("📋 セットアップ手順:")
    print("1. Google Driveの '001_PBL_チームD_AI4th' フォルダを開く")
    print("2. Firebase Consoleからダウンロードしたサービスアカウントキーを配置")
    print("3. ファイル名: toyasapo-firebase-adminsdk-fbsvc-93683c6f93.json")
    print("")
    print("完了したら、このセルを再実行してください。")
else:
    # Firebase初期化
    try:
        cred = credentials.Certificate(KEY_PATH)
        firebase_admin.initialize_app(cred)
        print("✅ Firebase初期化完了")
    except ValueError:
        print("✅ Firebase既に初期化済み")

    # Firestoreクライアント
    db = firestore.client()
    print("✅ Firestore接続完了")

✅ Firebase既に初期化済み
✅ Firestore接続完了


## 3. Firestoreからメールデータを取得

CSVアップロード不要！Firestoreから直接取得します。

In [ ]:
# Firestoreからメールデータを取得
print("📧 Firestoreからメールデータを取得中...")

# emailsコレクションから取得
emails_ref = db.collection('emails')
docs = emails_ref.stream()

# DataFrameに変換
data = []
for doc in docs:
    doc_data = doc.to_dict()
    # 削除されていないメールのみ
    if not doc_data.get('isDeleted', False):
        data.append(doc_data)

df = pd.DataFrame(data)

print(f"✅ データ取得完了: {len(df)}件")
print(f"\nカラム: {list(df.columns)}")

📧 Firestoreからメールデータを取得中...
✅ データ取得完了: 1104件

カラム: ['datetime', 'classification', 'id', 'inquiry', 'createdAt', 'response', 'updatedAt']


In [ ]:
# カラム名を特定
content_col = None
for col in ['inquiry', 'content', 'body', '本文', '本文(匿名化済)', 'Content', 'Body', 'message']:
    if col in df.columns:
        content_col = col
        break

date_col = None
for col in ['datetime', 'createdAt', 'date', 'receivedAt', 'timestamp', 'created_at', '日時', '日付']:
    if col in df.columns:
        date_col = col
        break

print(f"本文カラム: {content_col}")
print(f"日付カラム: {date_col}")
print(f"\n総件数: {len(df)}件")

本文カラム: inquiry
日付カラム: datetime

総件数: 1104件


## 4. キーワード分析（形態素解析）

In [ ]:
# 形態素解析器の初期化
tokenizer = Tokenizer()

# ストップワード（大幅拡充版）
stop_words = {
    # 一般的すぎる名詞
    'こと', 'もの', 'ため', 'よう', 'ところ', 'とき', '方', '人', '私', '自分',
    '今', '後', '前', '中', '上', '下', '件', '点', '旨', '等', '頃', '際',
    '度', '分', '回', '月', '日', '年', '時', '週', '先', '次', '別', '他',
    '以上', '以下', '以前', '以降', '場合', '状態', '状況', '理由', '原因',
    '確認', '質問', '相談', '依頼', '希望', '要望', '対応', '返答',
    '宛名', '冒頭', '文末', '本文', '件名', '内容', '下記', '上記',
    '可能', '必要', '大変', '承知', '了解', '申し訳', '恐れ入り',
    'こちら', 'そちら', '時間', '紹介', '変更',
    'あちら', 'どちら'

    # メール特有
    'メール', 'アドレス', '問い合わせ', '問合せ', '連絡', '返信', '送信',
    '宛先', 'お問い合わせ', 'フォーム', 'メールアドレス',

    # 敬語・挨拶
    'お世話', 'お願い', 'ご連絡', 'ご確認', 'ご質問', 'ご相談',
    'はじめまして', '初めまして', 'よろしく', '宜しく', '皆様', '皆さま',

    # クリニック固有（ノイズ対策）
    'toyano', 'mental', 'TMC', 'toya', 'とやの', 'メンタル', 'クリニック',
    'niigata', '新潟',

    # SNS・URL
    'facebook', 'twitter', 'instagram', 'LINE', 'line', 'http', 'https', 'www',

    # その他
    '名前', '削除', '担当', '者', '様', '氏', '殿', '御中', 'さん', '先生',
    '電話', '番号', '住所', '現在', '最近',
}

def extract_nouns(text):
    """テキストから名詞を抽出"""
    if pd.isna(text):
        return []
    nouns = []
    for token in tokenizer.tokenize(str(text)):
        if token.part_of_speech.startswith('名詞'):
            word = token.surface
            # フィルタリング条件
            if (len(word) >= 2 and
                word not in stop_words and
                not word.isascii() and  # 英数字のみは除外
                not word.isdigit() and  # 数字のみは除外
                not any(c in word for c in '[](){}「」【】、。・>=</:\"')):
                nouns.append(word)
    return nouns

if content_col:
    print("形態素解析中...")
    all_nouns = []
    for text in df[content_col]:
        all_nouns.extend(extract_nouns(text))

    noun_counts = Counter(all_nouns)
    print(f"✅ 完了: {len(all_nouns)}個の名詞を抽出")
else:
    print("⚠️ 本文カラムが見つかりません")

形態素解析中...
✅ 完了: 15288個の名詞を抽出


In [ ]:
# キーワードTOP20
top20 = noun_counts.most_common(20)

print("="*50)
print("🔑 キーワードTOP20（名詞のみ）")
print("="*50)
for i, (word, count) in enumerate(top20, 1):
    bar = '█' * (count // 30)
    print(f"{i:2}. {word:10} {count:4}件 {bar}")

🔑 キーワードTOP20（名詞のみ）
 1. 予約          783件 ██████████████████████████
 2. 受診          370件 ████████████
 3. 診察          358件 ███████████
 4. 病院          232件 ███████
 5. 診断          193件 ██████
 6. 障害          177件 █████
 7. カウンセリング     161件 █████
 8. 仕事          142件 ████
 9. 通院          133件 ████
10. 初診          129件 ████
11. 治療          117件 ███
12. 症状          116件 ███
13. 精神          116件 ███
14. 心療内科        114件 ███
15. 検査          105件 ███
16. キャンセル        97件 ███
17. 不安           94件 ███
18. 診療           92件 ███
19. お忙しい         81件 ██
20. 処方           76件 ██


In [ ]:
# TF-IDFでキーワード分析
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# テキストを名詞のみに変換する関数
def tokenize_to_nouns(text):
    nouns = extract_nouns(text)
    return ' '.join(nouns)

# 全メールを名詞のみのテキストに変換
texts = df[content_col].fillna('').apply(tokenize_to_nouns).tolist()

# TF-IDF計算
vectorizer = TfidfVectorizer(max_features=100)
tfidf_matrix = vectorizer.fit_transform(texts)

# 単語ごとの平均TF-IDFスコアを計算
feature_names = vectorizer.get_feature_names_out()
mean_tfidf = np.array(tfidf_matrix.mean(axis=0)).flatten()

# スコア順にソート
tfidf_ranking = sorted(zip(feature_names, mean_tfidf), key=lambda x: x[1], reverse=True)

print("=" * 50)
print("🎯 TF-IDF キーワードTOP20（重要度順）")
print("=" * 50)
for i, (word, score) in enumerate(tfidf_ranking[:20], 1):
    bar = '█' * int(score * 500)
    print(f"{i:2}. {word:10} {score:.4f} {bar}")

🎯 TF-IDF キーワードTOP20（重要度順）
 1. 予約         0.1740 ███████████████████████████████████████████████████████████████████████████████████████
 2. 診察         0.0879 ███████████████████████████████████████████
 3. 受診         0.0866 ███████████████████████████████████████████
 4. 初診         0.0489 ████████████████████████
 5. 病院         0.0466 ███████████████████████
 6. カウンセリング    0.0459 ██████████████████████
 7. 診断         0.0454 ██████████████████████
 8. 障害         0.0416 ████████████████████
 9. 心療内科       0.0395 ███████████████████
10. キャンセル      0.0382 ███████████████████
11. 仕事         0.0361 ██████████████████
12. 通院         0.0353 █████████████████
13. 精神         0.0315 ███████████████
14. 検査         0.0308 ███████████████
15. 診療         0.0269 █████████████
16. 症状         0.0265 █████████████
17. 治療         0.0251 ████████████
18. オンライン      0.0242 ████████████
19. 不安         0.0235 ███████████
20. 本日         0.0225 ███████████


In [ ]:
# N-gram（2語の組み合わせ）分析
from collections import Counter

def extract_bigrams(text):
    """テキストから2語の組み合わせを抽出"""
    nouns = extract_nouns(text)
    if len(nouns) < 2:
        return []
    bigrams = []
    for i in range(len(nouns) - 1):
        # 同じ単語の連続を除外
        if nouns[i] != nouns[i+1]:
            bigram = f"{nouns[i]}_{nouns[i+1]}"
            bigrams.append(bigram)
    return bigrams

# 全メールから2語の組み合わせを抽出
all_bigrams = []
for text in df[content_col]:
    all_bigrams.extend(extract_bigrams(text))

bigram_counts = Counter(all_bigrams)

print("=" * 50)
print("🔗 N-gram（2語の組み合わせ）TOP20")
print("=" * 50)
for i, (bigram, count) in enumerate(bigram_counts.most_common(20), 1):
    words = bigram.replace('_', ' → ')
    bar = '█' * (count // 5)
    print(f"{i:2}. {words:20} {count:4}件 {bar}")

🔗 N-gram（2語の組み合わせ）TOP20
 1. 発達 → 障害                55件 ███████████
 2. 初診 → 予約                45件 █████████
 3. オンライン → 予約             45件 █████████
 4. 診察 → 予約                43件 ████████
 5. 自立 → 支援                40件 ████████
 6. 新規 → 予約                37件 ███████
 7. 予約 → キャンセル             34件 ██████
 8. 障害 → 診断                33件 ██████
 9. 行動 → 療法                29件 █████
10. 受診 → 予約                29件 █████
11. 認知 → 行動                28件 █████
12. 適応 → 障害                27件 █████
13. 市民 → 病院                24件 ████
14. ネット → 予約               21件 ████
15. 医療 → 機関                21件 ████
16. コロナ → ワクチン             19件 ███
17. 次回 → 診察                19件 ███
18. 心理 → カウンセリング           16件 ███
19. カウンセリング → 予約           16件 ███
20. 心療内科 → 通院              16件 ███


In [34]:
# 共起分析（特定キーワードと一緒に出てくる単語）
from collections import defaultdict

def get_cooccurrence(texts, target_word, content_col):
    """特定の単語と一緒に出てくる単語をカウント"""
    cooccur = Counter()
    for text in texts:
        nouns = extract_nouns(text)
        if target_word in nouns:
            for noun in nouns:
                if noun != target_word:
                    cooccur[noun] += 1
    return cooccur

# 分析したいキーワード（形態素解析で分割されるものは短い方で）
target_words = ['うつ', '発達', '予約', '不安', '仕事', '休職']

print("=" * 60)
print("🔍 共起分析（キーワードと一緒に出てくる単語）")
print("=" * 60)

cooccurrence_results = {}

for target in target_words:
    cooccur = get_cooccurrence(df[content_col], target, content_col)
    cooccurrence_results[target] = cooccur.most_common(10)

    print(f"\n【{target}】と一緒に出てくる単語TOP10")
    for i, (word, count) in enumerate(cooccur.most_common(10), 1):
        bar = '█' * (count // 3)
        print(f"  {i:2}. {word:10} {count:3}件 {bar}")

🔍 共起分析（キーワードと一緒に出てくる単語）

【うつ】と一緒に出てくる単語TOP10
   1. 診断          31件 ██████████
   2. 受診          28件 █████████
   3. 本人          24件 ████████
   4. 行動          22件 ███████
   5. 病院          21件 ███████
   6. 義理          21件 ███████
   7. そう          21件 ███████
   8. 仕事          21件 ███████
   9. 心理          19件 ██████
  10. 時代          16件 █████

【発達】と一緒に出てくる単語TOP10
   1. 障害          72件 ████████████████████████
   2. 検査          32件 ██████████
   3. 診断          21件 ███████
   4. 病院          15件 █████
   5. メール         14件 ████
   6. 治療          10件 ███
   7. 受診           9件 ███
   8. 予約           9件 ███
   9. 診察           8件 ██
  10. 通院           8件 ██

【予約】と一緒に出てくる単語TOP10
   1. 受診         164件 ██████████████████████████████████████████████████████
   2. 診察         162件 ██████████████████████████████████████████████████████
   3. 病院         118件 ███████████████████████████████████████
   4. キャンセル       87件 █████████████████████████████
   5. 初診          81件 ███████████████████████████

## 5. 医療・メンタルヘルス キーワード分析

In [38]:
# キーワード辞書（拡充版）
keywords_dict = {
    '病名・診断': [
        # うつ系
        'うつ病', 'うつ', '鬱', '鬱病', '抑うつ', '大うつ',
        # 双極性
        '双極性障害', '双極性', '躁うつ', '躁鬱', '気分障害', '躁状態',
        # 不安系
        'パニック障害', 'パニック', '社交不安', '社会不安', '対人恐怖', '広場恐怖', '全般性不安',
        # 発達系
        '発達障害', 'ADHD', '注意欠陥', 'ASD', '自閉症', 'アスペルガー', '学習障害', 'LD',
        # 適応・ストレス
        '適応障害', 'PTSD', '心的外傷', 'トラウマ', '急性ストレス',
        # 睡眠
        '不眠症', '睡眠障害', '過眠', '睡眠時無呼吸',
        # 強迫・解離
        '強迫性障害', '強迫症', 'OCD', '解離性障害', '解離',
        # 摂食
        '摂食障害', '過食', '拒食', '過食症', '拒食症',
        # 依存
        '依存症', 'アルコール依存', 'ゲーム依存', 'ギャンブル依存',
        # パーソナリティ
        '人格障害', 'パーソナリティ障害', 'BPD', '境界性',
        # その他
        '統合失調症', '認知症', '更年期障害', '自律神経失調症',
        'PMS', 'PMDD', 'HSP', 'SAD', '季節性'
    ],
    '症状': [
        # 睡眠
        '眠れない', '不眠', '寝れない', '早朝覚醒', '中途覚醒', '過眠', '悪夢',
        # 不安・緊張
        '不安', '動悸', '息苦しい', '過呼吸', '緊張', '恐怖', 'パニック発作',
        # 気分
        '落ち込み', '憂うつ', 'やる気が出ない', '無気力', '虚しい', '希望がない',
        # 身体
        '頭痛', 'めまい', '吐き気', '食欲不振', '食欲がない', '倦怠感', 'だるい', '疲れ',
        '手の震え', '動悸', '胸が苦しい', '息切れ',
        # 思考・集中
        '集中できない', '物忘れ', '頭が回らない', '決断できない',
        # 感情
        '涙が止まらない', '泣いてしまう', 'イライラ', '怒り', '感情の起伏',
        # 対人
        '人が怖い', '外出できない', '引きこもり',
        # 深刻
        '希死念慮', '死にたい', '自傷', '消えたい', 'リストカット',
        # 解離・幻覚
        'フラッシュバック', '幻聴', '幻覚', '妄想', '離人感'
    ],
    '悩み・状況': [
        # 仕事
        '休職', '復職', '退職', '仕事', '職場', '上司', '同僚', '部下', '転職',
        'パワハラ', 'セクハラ', 'モラハラ', 'いじめ', '人間関係', '残業', '過労',
        # 家族
        '家族', '親', '母', '父', '夫', '妻', '子供', '息子', '娘',
        '育児', '介護', '離婚', '結婚', '嫁姑', '義母', '義父',
        # 恋愛・対人
        '恋愛', '恋人', '彼氏', '彼女', '友人', '孤独', '孤立',
        # 学校
        '学校', '不登校', '受験', '進路', '就活', '留年', '退学',
        # その他
        'ストレス', '引きこもり', '借金', '経済的', 'DV', '虐待'
    ],
    '相談内容': [
        # 予約
        '予約', 'キャンセル', '変更', '初診', '再診', '通院',
        # 書類
        '診断書', '紹介状', '意見書', '証明書',
        # 治療
        'セカンドオピニオン', '薬', '処方', '副作用', '減薬', '断薬',
        'カウンセリング', '心理検査', '知能検査', 'WAIS', '認知行動療法', 'CBT',
        # 制度・費用
        '費用', '料金', '保険', '自立支援', '障害年金', '傷病手当', '生活保護',
        '障害者手帳', '手帳',
        # その他
        '入院', '転院', '紹介', 'オンライン', 'リモート'
    ]
}
# 同義語辞書（統合するキーワード）
synonyms = {
    'うつ': ['うつ', 'うつ病', '鬱', '鬱病', '抑うつ', '大うつ'],
    '双極性障害': ['双極性障害', '双極性', '躁うつ', '躁鬱', '気分障害'],
    'パニック障害': ['パニック障害', 'パニック', 'パニック発作'],
    '発達障害': ['発達障害', 'ADHD', '注意欠陥', 'ASD', '自閉症', 'アスペルガー'],
    '不眠': ['不眠', '不眠症', '眠れない', '寝れない', '睡眠障害', '早朝覚醒', '中途覚醒'],
    '不安': ['不安', '不安感', '心配'],
    '適応障害': ['適応障害'],
    'PTSD': ['PTSD', '心的外傷', 'トラウマ'],
    '強迫性障害': ['強迫性障害', '強迫症', 'OCD'],
    '摂食障害': ['摂食障害', '過食', '拒食', '過食症', '拒食症'],
}

def count_keywords_with_synonyms(texts, category_keywords, synonyms_dict):
    """同義語を統合してカウント"""
    counts = Counter()

    for text in texts:
        if pd.isna(text):
            continue
        text = str(text)

        # 同義語辞書にあるものは統合
        counted_groups = set()
        for group_name, words in synonyms_dict.items():
            if group_name in counted_groups:
                continue
            for word in words:
                if word in text and word in category_keywords:
                    counts[group_name] += 1
                    counted_groups.add(group_name)
                    break

        # 同義語辞書にないものは個別カウント
        for keyword in category_keywords:
            in_synonym = any(keyword in words for words in synonyms_dict.values())
            if not in_synonym and keyword in text:
                counts[keyword] += 1

    return counts

# 各カテゴリの集計（同義語統合版）
results_unified = {}
for category, keywords in keywords_dict.items():
    counts = count_keywords_with_synonyms(df[content_col], keywords, synonyms)
    results_unified[category] = counts.most_common(10)

# 結果表示
print("=" * 60)
print("🏥 医療・メンタルヘルス キーワード分析（同義語統合版）")
print("=" * 60)

for category, top_items in results_unified.items():
    print(f"\n【{category}】")
    if top_items:
        for word, count in top_items[:5]:
            bar = '█' * (count // 5)
            print(f"  {word:15} {count:4}件 {bar}")
    else:
        print("  データなし")


def count_keywords(texts, keywords):
    counts = Counter()
    for text in texts:
        if pd.isna(text):
            continue
        text = str(text)
        for keyword in keywords:
            if keyword in text:
                counts[keyword] += 1
    return counts

# 各カテゴリの集計
results = {}
for category, keywords in keywords_dict.items():
    counts = count_keywords(df[content_col], keywords)
    results[category] = counts.most_common(10)

# 結果表示
print("="*60)
print("🏥 医療・メンタルヘルス キーワード分析")
print("="*60)

for category, top_items in results.items():
    print(f"\n【{category}】")
    if top_items:
        for word, count in top_items[:5]:
            bar = '█' * (count // 5)
            print(f"  {word:15} {count:4}件 {bar}")
    else:
        print("  データなし")

🏥 医療・メンタルヘルス キーワード分析（同義語統合版）

【病名・診断】
  うつ                60件 ████████████
  発達障害              57件 ███████████
  適応障害              27件 █████
  パニック障害            19件 ███
  双極性障害             11件 ██

【症状】
  不安                73件 ██████████████
  不眠                20件 ████
  動悸                18件 ███
  落ち込み              16件 ███
  頭痛                12件 ██

【悩み・状況】
  仕事                94件 ██████████████████
  母                 63件 ████████████
  夫                 59件 ███████████
  娘                 50件 ██████████
  親                 49件 █████████

【相談内容】
  予約               464件 ████████████████████████████████████████████████████████████████████████████████████████████
  薬                172件 ██████████████████████████████████
  紹介               123件 ████████████████████████
  初診               120件 ████████████████████████
  カウンセリング          114件 ██████████████████████
🏥 医療・メンタルヘルス キーワード分析

【病名・診断】
  発達障害              39件 ███████
  うつ                36件 ███████
  適応障害              27件 █████


## 6. サマリーレポート

In [ ]:
print("\n" + "="*60)
print("📋 分析サマリーレポート")
print("="*60)

print(f"\n■ データ概要")
print(f"  総問い合わせ件数: {len(df)}件")

print(f"\n■ キーワードTOP5")
for i, (word, count) in enumerate(noun_counts.most_common(5), 1):
    print(f"  {i}. {word} ({count}件)")

print(f"\n■ 病名・診断TOP3")
for word, count in results['病名・診断'][:3]:
    print(f"  - {word}: {count}件")

print(f"\n■ 症状TOP3")
for word, count in results['症状'][:3]:
    print(f"  - {word}: {count}件")

print(f"\n■ 悩み・状況TOP3")
for word, count in results['悩み・状況'][:3]:
    print(f"  - {word}: {count}件")

print(f"\n■ 相談内容TOP3")
for word, count in results['相談内容'][:3]:
    print(f"  - {word}: {count}件")

print("\n" + "="*60)


📋 分析サマリーレポート

■ データ概要
  総問い合わせ件数: 1104件

■ キーワードTOP5
  1. 予約 (783件)
  2. 受診 (370件)
  3. 診察 (358件)
  4. 病院 (232件)
  5. 診断 (193件)

■ 病名・診断TOP3
  - 発達障害: 39件
  - うつ: 36件
  - 適応障害: 27件

■ 症状TOP3
  - 不安: 73件
  - 落ち込み: 16件
  - 不眠: 13件

■ 悩み・状況TOP3
  - 仕事: 94件
  - 夫: 59件
  - 親: 49件

■ 相談内容TOP3
  - 予約: 464件
  - 薬: 172件
  - 初診: 120件



## 7. 🔥 Firestoreに保存

In [39]:
# 分析結果をFirestoreに保存（全機能版）
analysis_data = {
    'updated_at': datetime.now().isoformat(),
    'total_count': len(df),

    # キーワードTOP20（単純カウント）
    'keywords_top20': [
        {'word': word, 'count': count}
        for word, count in noun_counts.most_common(20)
    ],

    # TF-IDF TOP20（重要度順）
    'keywords_tfidf': [
        {'word': word, 'score': round(float(score), 4)}
        for word, score in tfidf_ranking[:20]
    ],

    # N-gram TOP20（2語の組み合わせ）
    'bigrams_top20': [
        {'words': bigram.replace('_', ' → '), 'count': count}
        for bigram, count in bigram_counts.most_common(20)
    ],

    # 共起分析
    'cooccurrence': {
        target: [
            {'word': word, 'count': count}
            for word, count in words[:10]
        ]
        for target, words in cooccurrence_results.items()
    },

    # 病名・診断（同義語統合版）
    'disease': [
        {'word': word, 'count': count}
        for word, count in results_unified['病名・診断'][:10]
    ],

    # 症状（同義語統合版）
    'symptoms': [
        {'word': word, 'count': count}
        for word, count in results_unified['症状'][:10]
    ],

    # 悩み・状況（同義語統合版）
    'concerns': [
        {'word': word, 'count': count}
        for word, count in results_unified['悩み・状況'][:10]
    ],

    # 相談内容（同義語統合版）
    'inquiry_types': [
        {'word': word, 'count': count}
        for word, count in results_unified['相談内容'][:10]
    ],
}

# Firestoreに保存
db.collection('analysis_results').document('latest').set(analysis_data)

print("✅ 分析結果をFirestoreに保存しました！")
print(f"\n保存先: analysis_results/latest")
print(f"更新日時: {analysis_data['updated_at']}")
print(f"\n保存内容:")
print(f"  - キーワードTOP20")
print(f"  - TF-IDF TOP20")
print(f"  - N-gram TOP20")
print(f"  - 共起分析（{len(cooccurrence_results)}キーワード）")
print(f"  - 病名・症状・悩み・相談内容（同義語統合版）")
print("\n🎉 ToyaSapoのダッシュボードで表示できます！")

✅ 分析結果をFirestoreに保存しました！

保存先: analysis_results/latest
更新日時: 2025-12-13T14:46:04.005650

保存内容:
  - キーワードTOP20
  - TF-IDF TOP20
  - N-gram TOP20
  - 共起分析（6キーワード）
  - 病名・症状・悩み・相談内容（同義語統合版）

🎉 ToyaSapoのダッシュボードで表示できます！


---

## 🎉 完了！

### 次回からの使い方
1. このノートブックを開く
2. 「ランタイム」→「すべてのセルを実行」
3. Google Driveのアクセス許可を承認
4. 完了！

### 保存されるデータ
- `analysis_results/latest`
  - `updated_at`: 更新日時
  - `total_count`: 総件数
  - `keywords_top20`: キーワードTOP20
  - `disease`: 病名・診断
  - `symptoms`: 症状
  - `concerns`: 悩み・状況
  - `inquiry_types`: 相談内容